In [ ]:
from comet import download_model, load_from_checkpoint

# Import CometKiwi Model
model_path = download_model("Unbabel/wmt22-cometkiwi-da")
model = load_from_checkpoint(model_path)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.2 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../.cache/huggingface/hub/models--Unbabel--wmt22-cometkiwi-da/snapshots/b3a8aea5a5fc22db68a554b92b3d96eb6ea75cc9/checkpoints/model.ckpt`
/fs/classhomes/fall2024/cmsc723/c7230021/miniconda3/lib/python3.12/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [ ]:
from torch import nn
from torchtune.modules.peft import LoRALinear

def replace_qlora(module, name="<ROOT>"):
    '''
    Replace Linear layers with LoRALinear layers, recursively.
    '''
    for attr_str, _ in module.named_children():
        target_attr = getattr(module, attr_str)
        if type(target_attr) == nn.Linear and "lora" not in attr_str:
            #print('replacing: ', name, attr_str)
            try:
                new_ql = nn.Sequential(LoRALinear(in_dim=target_attr.in_features, out_dim=target_attr.out_features, rank=min([target_attr.in_features, target_attr.out_features, 4]), alpha=0.1, quantize_base=True), 
                           BiasLayer(dim=target_attr.out_features))

                setattr(module, attr_str, new_ql)
            except:
                print(name, attr_str, "failed, check dims:", target_attr.in_features, target_attr.out_features)

    for name, immediate_child_module in module.named_children():
        replace_qlora(immediate_child_module, name)

replace_qlora(model)

In [6]:
from torch import randn
class BiasLayer(torch.nn.Module):
    """A layer that simply adds a trainable bias vector to an input vector."""
    def __init__(self, dim) -> None:
        super().__init__()
        bias_value = randn((dim))
        self.bias_layer = nn.Parameter(bias_value)
    
    def forward(self, x):
        return x + self.bias_layer